# Omówienie zbioru danych - EDA

Dane zawierają informacje na temat wypożyczeń rowerów

Źródło: https://www.kaggle.com/code/sasakitetsuya/analysis-for-bike-use-in-helsinki

Ogólny cel projektu:
1. Zrozumienie danych i analizę różnych aspektów takich jak, charakterystyka stacji czy klientów.
2. Obróbka danych - jest ich bardzo dużo i należy je odpowiednio przygotować.
3. Model prognozy przyszłorocznej liczby wypożyczeń.
4. Prognoza wypożyczeń na kolejny dzień.
5. System do alertów (progonozowanie czy wypożyczeń będzie więcej niż zwrotów) w celu szybszej reakcji.


Cele EDA:
Dane te pozwalają na analizę wielu aspektów. 
Lista zadań do realizacji:
1. Pobranie danych i podstawowe statystyki (head, info)
2. Konwersja dat. (z formatu object na datetime)
3. Agregacja danych:
    - Przygotowanie funkcji do agregacji
    - Stworzenie i zapisanie ramki danych z sumą wypożyczeń, średnim dystansem, prędkością i pogodą dla każdego dnia.
    - Stworzenie i zapisanie ramki danych z sumą wypożyczeń, średnim dystansem, prędkością i pogodą dla każdego dnia i stacji.
    - Zapisanie zagregowanych danych.
4. Mapa stacji
    - Stworzenie wykresu położenia stacji. (w zbiorze mamy współrzędne geograficzne).
5. Analiza stacji:
    - minimalne i maksymalne daty wypożyczeń dla stacji
    - liczba wypożyczeń (Najpopularniejsze stacje)
6. Analiza użytkowników:
- Jaki jest rozkład:
    - pokonywanej odległości
    - czasu
    - prędkości
- Czy zmienia się po latach?
7. Funkcja do oceny czy w pobliżu jest stacja
- Funkcja, która dla wybranej stacji, sprawdzi w jakiej odległości znajduje się najbliższa stacja
- Pozwoli dokonać ostatecznej oceny, czy stacje z małą liczbą wypożyczeń można usunąć.





In [ ]:
#pip install kagglehub

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

1. Pobranie danych i podstawowe statystyki (head, info)

In [ ]:
import kagglehub


In [ ]:
# wczytanie danych


In [ ]:
# info 


In [ ]:
# head


2. Konwersja dat. (z formatu object na datetime)

In [ ]:

df['departure'] = pd.to_datetime(df['departure'])
df["departure_date"] = df['departure'].dt.round("D")
df["departure_date_hours"] = df['departure'].dt.round("h")

In [ ]:
df['return'] = pd.to_datetime(df['return'])
df["return_date"] = df['return'].dt.round("D")
df["return_date_hours"] = df['return'].dt.round("h")

In [ ]:
df.to_parquet('data/source_file.parquet')

3. Agregacja danych:
    - Przygotowanie funkcji do agregacji
    - Stworzenie i zapisanie ramki danych z sumą wypożyczeń, średnim dystansem, prędkością i pogodą dla każdego dnia.
    - Stworzenie i zapisanie ramki danych z sumą wypożyczeń, średnim dystansem, prędkością i pogodą dla każdego dnia i stacji.
    - Zapisanie zagregowanych danych.

In [ ]:
import os

In [ ]:
from help_function import agg_data

In [ ]:
## ogólny szereg czasowy - liczba wypożyczeń dziennie
df_total_agg = agg_data(df,['departure_date'],
                       {'departure_name':'count',
                       'distance (m)': 'mean',
                       'duration (sec.)': 'mean',
                       'avg_speed (km/h)': 'mean',
                       'Air temperature (degC)': 'mean'})


In [ ]:
df_total_agg

In [ ]:
#os.mkdir('data')

In [ ]:
df_total_agg.rename(columns={'departure_name':'numbers_of_renting'}).to_parquet('data/total_agg.parquet')

In [ ]:
df_total_agg.head()

In [ ]:
## aggregate data - departures
df_agg_dep= agg_data(df,
                    ['departure_id','departure_date_hours'],
                    {'departure_name':'count',
                    'Air temperature (degC)': 'mean',
                    'distance (m)': 'mean',
                    'duration (sec.)':'mean'}  )

In [ ]:
df_agg_dep = df_agg_dep.rename(columns={'departure_name': 'nr_of_departures'})

In [ ]:
df_agg_dep.to_parquet('data/hourly_data_per_station.parquet')

In [ ]:
df_agg_dep.head()

In [ ]:
## aggregate data - departures
df_agg_ret= agg_data(df,
                    ['return_id','return_date_hours'],
                    {'return_name':'count',
                    'Air temperature (degC)': 'mean',
                    'distance (m)': 'mean',
                    'duration (sec.)':'mean'}  )

In [ ]:
df_agg_ret = df_agg_ret.rename(columns={'return_name':'nr_of_returns'})

In [ ]:
df_agg_ret.to_parquet('data/hourly_data_per_station_returns.parquet')

In [ ]:
check = df_agg_ret.groupby(['return_id','return_date_hours']).count()[['nr_of_returns']]

In [ ]:
check[check['nr_of_returns']>1]

5. Mapa stacji
    - Stworzenie wykresu położenia stacji. (w zbiorze mamy współrzędne geograficzne).

In [ ]:
pip install folium

In [ ]:
df.head()

In [ ]:
df_map = df.loc[df['departure']>='2020-01-01',[
    'departure_name','departure_latitude', 'departure_longitude','return_name']].groupby(
        'departure_name').agg({'departure_latitude': 'mean',
                              'departure_longitude': 'mean',
                              'return_name': 'count'
                              }).reset_index().rename(columns={'return_name':'amount'})

In [ ]:
df_map

In [ ]:
import folium

In [ ]:
the_map= folium.Map(location=[df_map.departure_latitude.mean(), df_map.departure_longitude.mean()], zoom_start=10)

In [ ]:
for i in range(len(df_map)):
    lat = df.loc[i,'departure_latitude']
    lon = df.loc[i,'departure_longitude']
    amount = df_map.loc[i,'amount']
    name= df_map.loc[i,'departure_name']
    folium.Marker(location=[lat, lon], popup=f"Nazwa: {name}\n wypoczenia: {amount}",).add_to(the_map)

In [ ]:
the_map

In [ ]:
the_map_2= folium.Map(location=[df_map.departure_latitude.mean(), df_map.departure_longitude.mean()], zoom_start=10)

In [ ]:
for i in range(len(df_map)):
    lat = df_map.loc[i,'departure_latitude']
    lon = df_map.loc[i,'departure_longitude']
    amount = df_map.loc[i,'amount']
    name= df_map.loc[i,'departure_name']
    folium.CircleMarker(
        location=[lat, lon],
        radius=amount / 1000,  # Skala wielkości
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=f"Liczba wypożyczeń: {amount}\n stacja: {name}"
    ).add_to(the_map_2)

In [ ]:
the_map_2

In [ ]:
# zapisywanie mapy
the_map_2.save("map.html")

5. Analiza stacji:
    - minimalne i maksymalne daty wypożyczeń dla stacji
    - liczba wypożyczeń (Najpopularniejsze stacje)

In [ ]:

# minimalna data wypożyczenia ze stacji
min_dates = df.loc[:,['departure_id','departure_date']].groupby('departure_id').min()
df['departure_date'].min()
df['departure_date'].max()
min_dates[min_dates['departure_date']>='2020-01-01']

In [ ]:
df_map.describe()

In [ ]:
potential_reduction = df_map[df_map['amount']<=df_map.amount.quantile(0.25)]
potential_reduction

6. Analiza użytkowników:
- Jaki jest rozkład:
    - pokonywanej odległości
    - czasu
    - prędkości
- Czy zmienia się po latach?

In [ ]:
df.head()

In [ ]:
sns.kdeplot(df,x='distance (m)')

In [ ]:
df[['distance (m)','duration (sec.)']].describe()

7. Funkcja do oceny czy w pobliżu jest stacja
- Funkcja, która dla wybranej stacji, sprawdzi w jakiej odległości znajduje się najbliższa stacja
- Pozwoli dokonać ostatecznej oceny, czy stacje z małą liczbą wypożyczeń można usunąć.

In [ ]:
pip install geopy

In [ ]:
from geopy.distance import geodesic

In [ ]:
def calculate_min_distance(data_point, points_to_check):
    distances = []
    for i in points_to_check:
        point = points_to_check[i]
        distance= geodesic(data_point, point).meters
        distances.append(distance)
    return min(distances), points_to_check.index()